# Welcome!

We appreciate you attending our DataJoint workshop. Let's learn how easy it is to create a DataJoint computational pipeline from scratch.

# Setup

Please run the below but feel free to disregard it as it is just some environment specific setup configuration.

In [ ]:
!cd /home/notebooks/notebook
import datajoint as dj
dj.config['database.password'] = None

In [ ]:
from datetime import date
from requests import request
import json
from os import path
from bs4 import BeautifulSoup
# , mkdir
# from datetime import datetime

# Workshop

## Problem Statement

- You are a journalist at UH covering sporting events around campus
- There are various sports for both men and women atheletes
- Each sporting event needs an article piece created on a specific date to promote and cover the event
- An article should include both a headline and a thumbnail image
- To promote both the article + sport, we should come up with an *attention-grabbing* flyer based on the article's thumbnail

## Getting Connected

Let's start by importing `datajoint`.

In [ ]:
import datajoint as dj

Next, let's open a connection to a database server instance or sometimes referred to as a pipeline. This step is not necessary but useful in debugging or simply verifying you have access.

In [ ]:
dj.conn()

We can list the databases you have access to by running:

In [ ]:
dj.list_schemas()

## Creating a Schema

Now let's create a database or sometimes referred to as a database (in MySQL). This will be the primary namespace where we will  create our pipeline.

In [ ]:
schema = dj.Schema('{user}_uh-showcase'.format(user=dj.config['database.user']))
schema

## Designing your Pipeline

From the description of the problem, we can make the following assumptions:
- A `Sport` has an attribute that determines whether it is for `men` or `women`
- `Sport:NewsArticle` should have a `1:M` relationship
- A `NewsArticle` has an attribute to specify when it was created
- A `NewsArticle` has a `1:1` relationship both for its headline and thumbnail

In [ ]:
@schema
class Sport(dj.Manual):
    definition = """
    sport_id  : int auto_increment
    ---
    name      : varchar(30)
    sex="men" : enum("women", "men")
    unique index(name, sex)
    """

In [ ]:
@schema
class NewsArticle(dj.Manual):
    definition = """
    -> Sport
    news_id   : int
    ---
    date=null : date
    """

To help visualize our pipeline, we can use the `dj.Diagram` function or `dj.Di` for short.

In [ ]:
dj.Di(schema)

## Inserting Records

Let's initialize some sports.

In [ ]:
Sport.insert([
    dict(name='swim', sex='women'),
    dict(name='soc', sex='women')
])

Let's also leverage the default value feature.

In [ ]:
Sport.insert([
    dict(name='track'),
    dict(name='football')
])

To confirm our inserts, we can preview the table's contents by running the following:

In [ ]:
Sport()

Now let's add some entries into for our news articles that correspond to actual articles from UH's atheletics site: https://uhcougars.com/.

In [ ]:
NewsArticle.insert([
    [2, 1, date(2019,11,5)],
    [1, 1, date(2020,3,11)],
    [3, 1, date(2020,3,4)],
])

In [ ]:
NewsArticle()

## Querying

Suppose now that we wanted to perform some query to our pipeline to extract data from it. The following are a few examples how we can achieve this.

Query1: `women` sports

In [ ]:
Sport & dict(sex='women')

Query2: `men` sports

In [ ]:
q = Sport & "sex='men'"
q

Query3: Exclude any `men` sports

In [ ]:
q = Sport - dict(sex='men')

`q` here is actually just a query expression and unevaluated. Think of `q` as a query builder. The only reason why the queries were displaying values before is because we have been utilizing Jupyter's preview feature (which limits the results). It is important to understand that queries aren't expected to actually run until we can `fetch` on them (more on this later).

In [ ]:
isinstance(q, dj.expression.QueryExpression)

Here we can manually ask Jupyter to preview the query for us.

In [ ]:
q

A great debugging tool as well is `make_sql` which allows us to see the assembled SQL query.

In [ ]:
q.make_sql()

Query4: Articles for `women`'s sports

In [ ]:
NewsArticle()

In [ ]:
q = Sport * NewsArticle & "sex = 'women'"
q

To properly run and fetch your query, you'd call `fetch` on your query expression like so:

In [ ]:
q.fetch()

In [ ]:
q.fetch(as_dict=True)

In [ ]:
q.fetch('name', 'sex', as_dict=True)

## Computation

In DataJoint, computation is a native part of interacting with your data. Let's see how to leverage this within our pipeline.

First, some important principles:
- There are 2 different kinds of computed tables in DataJoint: `Imported` and `Computed`.
- They operate essentially the same but we differentiate them purely on convention and how they are used.
- `Imported` tables signal that there is external data being introduced to the pipeline.
- `Computed` tables signal that there is new data being introduced but sourced from within the pipeline itself or sometimes referred to as upstream.

So we can include this in our design by creating an `Imported` table that runs a web scraping computation to fetch the headline and thumbnail for the articles. Then, we can create a `Computed` table that runs a computation to modify our thumbnail to make it more interesting.

Let's start first with our `Imported` table. We can make the following assumptions:
- The `Headline` table will store both the headline and the thumbnail for an article's given date.
- A record in the `Headline` table should correspond to a single news article i.e. `1:1` relationship.

In [ ]:
@schema
class Headline(dj.Imported):
    definition = """
    -> NewsArticle
    ---
    name : varchar(200)
    image: longblob
    """

    def make(self, key):    
        sport_name, sport_sex = (Sport & key).fetch1('name', 'sex')
        news_date = (NewsArticle & key).fetch1('date')
        file_path = 'cached/{}_{}_{}.jpg'.format(sport_name, sport_sex, news_date)

        if not path.exists(file_path):
            base_url = "https://uhcougars.com"
            headers = {
                'User-Agent': "DataJoint"
            }
            querystring = {
                "index":"1",
                "page_size":"200",
                "sport": sport_name if sport_sex != 'women' else sport_sex[0] + sport_name,
                "season":"0"
            }
            response = request("GET", base_url + "/services/archives.ashx/stories",
                               headers=headers, params=querystring)
            article = ([v for v in json.loads(response.text)['data']
                        if v['story_postdate'] == news_date.strftime("%-m/%-d/%Y")][0]
                       if news_date else json.loads(response.text)['data'][0])
            news_date = (datetime.strptime(article['story_postdate'],
                                           '%m/%d/%Y').date()
                         if not news_date else news_date)

            htmldata = request("GET", base_url + article['story_path'], headers=headers).text
            soup = BeautifulSoup(htmldata, 'html.parser')
            image_path = [i['src'] for i in soup.find_all('img') if i.has_attr('src')][2]
            response = request("GET", image_path, headers=headers)
            if not path.exists(path.dirname(file_path)):
                mkdir(path.dirname(file_path))
            with open(file_path, 'wb') as f:
                f.write(response.content)
            # print('Image `{}` downloaded.'.format(file_path))
            key['name'] = article['story_headline']
        else:
            key['name'] = 'No Title (loaded from cache)'
            # print('Image `{}` read from cache.'.format(file_path))

        with open(file_path, mode='rb') as f:
            key['image'] = f.read()
        self.insert1(key)

Let's see how this has updated our entity relationships:

In [ ]:
dj.Di(schema)

Let's check if there is any data yet in our new `Headline` table.

In [ ]:
Headline()

Our `Imported` table is ready to synchronize with the upstream but it is waiting for us to tell it to run the computations or populate itself. To do this, we can simply call the `populate` method on our `Headline` table. After running it, we can call populate as many times as we want after this since DataJoint will only process new entries upstream.

In [ ]:
Headline.populate(display_progress=True)

# Teardown

# Welcome to the UH workshop environment!

## Workshop setup

In [ ]:
!cd /home/notebooks/notebook
import datajoint as dj
dj.config['database.password'] = None

## Workshop

### Connecting to your data

In [ ]:
import datajoint as dj

### Open connection to a database server instance i.e. Pipeline

In [ ]:
dj.conn()

### List databases you have access to

In [ ]:
dj.list_schemas()

### Problem statement

- You are a journalist at UH covering sporting events around campus
- There are various sports for both men and women atheletes
- Each sporting event needs a article piece to promote cover the event
- This article will include both a headline and a thumbnail image
- To promote both the article + sport, we should come up with an attention grabbing flyer of the article thumbnails

# Structuring your data using a database i.e. Schema

In [ ]:
schema = dj.Schema('{user}_uh-showcase'.format(user=dj.config['database.user']))
schema

In [ ]:
dj.Diagram(schema)

In [ ]:
@schema
class Sport(dj.Manual):
    definition = """
    sport_id        :  int auto_increment
    ---
    name      : varchar(30)
    sex="men" : enum("women", "men")
    unique index(name, sex)
    """

In [ ]:
Sport.insert([
    dict(name='swim', sex='women'),
    dict(name='soc', sex='women')
])
Sport.insert([
    dict(name='track'),
    dict(name='football')
])

In [ ]:
Sport()

In [ ]:
Sport & dict(sex='women')

In [ ]:
q = Sport & "sex='men'"
q

In [ ]:
q.fetch(order_by='name DESC', format='frame')

In [ ]:
q.fetch()

In [ ]:
Sport - dict(sex='men')

In [ ]:
(Sport - dict(sex='men')).make_sql()

In [ ]:
from datetime import date

In [ ]:
@schema
class NewsArticle(dj.Manual):
    definition = """
    -> Sport
    news_id       : int
    ---
    date=null: date
    """

In [ ]:
dj.Di(schema)

In [ ]:
NewsArticle.insert([
    # [4, 2, date(2020,3,3)],
    [2, 2, date(2019,11,5)],
    [1, 2, date(2020,3,11)],
    [3, 2, date(2020,3,4)],
])
# NewsArticle.insert([
#     dict(sport_id=1, news_id=1),
#     dict(sport_id=3, news_id=1)
# ])
# NewsArticle.insert([
#     [4, 1, date(2021,1,29)],
#     [2, 1, date(2021,4,9)],
#     [1, 1, date(2021,3,20)],
#     [3, 1, date(2021,4,8)],
# ])

In [ ]:
NewsArticle()

In [ ]:
Sport * NewsArticle

In [ ]:
from requests import request
import json
from os import path, mkdir
from bs4 import BeautifulSoup
from datetime import datetime

In [ ]:
@schema
class Headline(dj.Imported):
    definition = """
    -> NewsArticle
    ---
    name : varchar(200)
    image: longblob
    """
    def make(self, key):    
        sport_name, sport_sex = (Sport & key).fetch1('name', 'sex')
        news_date = (NewsArticle & key).fetch1('date')
        file_path = 'cached/{}_{}_{}.jpg'.format(sport_name, sport_sex, news_date)

        if not path.exists(file_path):
            base_url = "https://uhcougars.com"
            headers = {
                'User-Agent': "DataJoint"
            }
            querystring = {
                "index":"1",
                "page_size":"200",
                "sport": sport_name if sport_sex != 'women' else sport_sex[0] + sport_name,
                "season":"0"
            }
            response = request("GET", base_url + "/services/archives.ashx/stories",
                               headers=headers, params=querystring)
            article = ([v for v in json.loads(response.text)['data']
                        if v['story_postdate'] == news_date.strftime("%-m/%-d/%Y")][0]
                       if news_date else json.loads(response.text)['data'][0])
            news_date = (datetime.strptime(article['story_postdate'],
                                           '%m/%d/%Y').date()
                         if not news_date else news_date)

            htmldata = request("GET", base_url + article['story_path'], headers=headers).text
            soup = BeautifulSoup(htmldata, 'html.parser')
#             print([i['src'] for i in soup.find_all('img') if i.has_attr('src')])
            image_path = [i['src'] for i in soup.find_all('img') if i.has_attr('src')][2]
            response = request("GET", image_path, headers=headers)
            if not path.exists(path.dirname(file_path)):
                mkdir(path.dirname(file_path))
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print('Image `{}` downloaded.'.format(file_path))
            key['name'] = article['story_headline']
        else:
            key['name'] = 'No Title (loaded from cache)'
            print('Image `{}` read from cache.'.format(file_path))

        with open(file_path, mode='rb') as f:
            key['image'] = f.read()
        self.insert1(key)

In [ ]:
dj.Di(schema)

In [ ]:
Headline()

In [ ]:
Headline.populate(display_progress=True)

In [ ]:
q_original = Headline()
q_original = Headline & dict(sport_id=1, news_id=2)
q_original

In [ ]:
from PIL import Image
from io import BytesIO
from matplotlib import pyplot as plt

In [ ]:
image_original = q_original.fetch1('image')
image_original = Image.open(BytesIO(image_original))
fig, axarr = plt.subplots(1,1,figsize=(15,15))
axarr.imshow(image_original)

In [ ]:
@schema
class PaintingStyle(dj.Lookup):
    definition = """
    style_name      : varchar(30)
    """
    contents = [
        ['udnie']
    ]

In [ ]:
dj.Di(schema)

In [ ]:
PaintingStyle.insert([
    dict(style_name='mosaic')
])

In [ ]:
PaintingStyle()

In [ ]:
import numpy as np
import cv2
import imutils

In [ ]:
@schema
class Flyer(dj.Computed):
    definition = """
    -> Headline
    -> PaintingStyle
    ---
    image: longblob
    """
    def make(self, key):
        style_path = 'models/' + key['style_name'] + '.t7'
        image = (Headline & key).fetch1('image')
        
        net = cv2.dnn.readNetFromTorch(style_path)
        image = np.frombuffer(image, np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        image = imutils.resize(image, width=600)
        (h, w) = image.shape[:2]

        # construct a blob from the image, set the input, and then perform a
        # forward pass of the network
        blob = cv2.dnn.blobFromImage(image, 1.0, (w, h),
            (103.939, 116.779, 123.680), swapRB=False, crop=False)
        net.setInput(blob)
        output = net.forward()

        # reshape the output tensor, add back in the mean subtraction, and
        # then swap the channel ordering
        output = output.reshape((3, output.shape[2], output.shape[3]))
        output[0] += 103.939
        output[1] += 116.779
        output[2] += 123.680
        output = output.transpose(1, 2, 0)
        output = np.clip(output, 0, 255)
        output= output.astype('uint8')
        
        print('sport_id: {sport_id}, news_id: {news_id}, style_name: {style_name}'.format(**key))
        
        key['image'] = cv2.imencode('.jpg', output)[1].tobytes()
        self.insert1(key)

In [ ]:
dj.Di(schema)

In [ ]:
Flyer.populate()

In [ ]:
q_styled = Flyer()
# q_styled = Flyer & dict(sport_id=1, news_id=2, style_name='udnie')
q_styled = Flyer & dict(sport_id=1, news_id=2, style_name='mosaic')
q_styled

In [ ]:
image_styled = q_styled.fetch1('image')
image_styled = Image.open(BytesIO(image_styled))

fig, axarr = plt.subplots(1,2,figsize=(15,15))
axarr[0].imshow(image_original)
axarr[1].imshow(image_styled)

In [ ]:
image_styled.save("./image_styled.jpg", 'jpeg')

# Clean up and remove generated data

In [ ]:
schema.drop()

In [ ]:
import shutil
shutil.rmtree('cached')

In [ ]:
import os
os.unlink("./image_styled.jpg")